In [1]:
import os
from numpy import load
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.neighbors import KDTree
import time

In [5]:
feature_list = load(r'npy\paris_feature_list.npy')
feature_id = load(r'npy\paris_feature_id.npy')
feature_id = feature_id.tolist()

model = EfficientNetB7(include_top=False)
kd = KDTree(feature_list)

In [7]:
def feature_extract(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    flattened_features = features.flatten()
    normalized_features = flattened_features / np.linalg.norm(flattened_features)
    return normalized_features.reshape(-1,1).T[0]

In [10]:
pathY = 'media\paris_dataset\dataY\paris'
query_files = os.listdir(pathY)

In [14]:
all_times = 0
for i in range(3, len(query_files), 4):
    query = open(pathY + '\\' + query_files[i]).read()[:-1].split(' ')
    x = int(float(query[1]))
    y = int(float(query[2]))
    w = int(float(query[3]))
    h = int(float(query[4]))
    good_results = open(pathY + '\\' + query_files[i-3]).readlines()
    ok_results = open(pathY + '\\' + query_files[i-1]).readlines()
    #junk_results = open(pathY + '\\' + query_files[i-2]).readlines()

    accept_results = good_results + ok_results #+ junk_results
    accept_results = [i[:-1] for i in accept_results]

    print('Query:', query[0])

    img = cv2.imread('media\paris_dataset\dataX\paris' + '\\' + query[0].split('_')[1] + '\\' + query[0] + '.jpg')
    cv2.imwrite('cut\\' + query[0] + '_cut.jpg', img[y:y+h, x:x+w])

    start = time.time()
    dist, indx = kd.query(feature_extract('cut\\' + query[0] + '_cut.jpg', model).reshape(-1,1).T, len(feature_list))
    end = time.time()
    all_times = all_times + (end-start)

    index = []
    distance = []
    label = []
    for j in range(len(feature_list)):
        index.append(j+1)
        distance.append(dist[0][j])
        label.append(1) if feature_id[indx[0][j]].split('/')[-1][:-4] in accept_results else label.append(0)

    df = pd.DataFrame(data={"index": index, "distance": distance, "label": label})
    df.to_csv('retrieval_query_results_paris/query_' + query[0] + '.csv', index=False)

Query: paris_defense_000605
1/1 [==============================] - 4s 4s/step
Query: paris_defense_000331
1/1 [==============================] - 0s 239ms/step
Query: paris_defense_000216
1/1 [==============================] - 0s 250ms/step
Query: paris_defense_000056
1/1 [==============================] - 0s 249ms/step
Query: paris_defense_000254
1/1 [==============================] - 0s 290ms/step
Query: paris_general_002985
1/1 [==============================] - 0s 265ms/step
Query: paris_general_001729
1/1 [==============================] - 0s 276ms/step
Query: paris_eiffel_000266
1/1 [==============================] - 0s 248ms/step
Query: paris_general_002645
1/1 [==============================] - 0s 246ms/step
Query: paris_general_002391
1/1 [==============================] - 0s 244ms/step
Query: paris_invalides_000355
1/1 [==============================] - 0s 241ms/step
Query: paris_invalides_000072
1/1 [==============================] - 0s 252ms/step
Query: paris_invalides_00049

In [15]:
all_times

80.26696157455444